In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

# Data dir
raw_data_dir = "../data/mimic/raw/"
processed_data_dir = "../data/mimic/processed/"

# Read csv files
pat = pd.read_csv(raw_data_dir + "patients.csv")
adm = pd.read_csv(raw_data_dir + "admissions.csv")
proc = pd.read_csv(raw_data_dir + "procedures_icd.csv")
diag = pd.read_csv(raw_data_dir + "diagnoses_icd.csv")
ie = pd.read_csv(raw_data_dir + "icustays.csv")
d_icd_proc = pd.read_csv(raw_data_dir + "d_icd_procedures.csv")
d_icd_diag = pd.read_csv(raw_data_dir + "d_icd_diagnoses.csv")

#processed_mimic = pd.read_csv(processed_data_dir + "sequential_mimic.csv")

/share/apps/anaconda-ci/fgci-centos7-anaconda/software/anaconda/2021-03-tf2/e54ecce8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
adm["admittime"] = pd.to_datetime(adm["admittime"])
adm["dischtime"] = pd.to_datetime(adm["dischtime"])

diag['icd_version'] = diag['icd_version'].astype(int)
proc['icd_version'] = proc['icd_version'].astype(int)

# Inner join to get demographics
demo = pd.merge(pd.merge(pat,adm,on='subject_id'),ie,on=['subject_id', 'hadm_id'])

# Assign age, all patients are older than 18 years old
demo = demo.assign(admission_age = demo['admittime'].dt.year - demo['anchor_year'] + demo['anchor_age'])
demo = demo.assign(death = demo['hospital_expire_flag'])

# Merge with procedure and diagnosis
demo = pd.merge(demo, diag, on=['subject_id', 'hadm_id'])
demo = demo.rename(columns={"icd_code": "diag_icd_code", "icd_version": "diag_icd_version", "seq_num": "diag_seq_num"})
demo = pd.merge(demo, proc, on=['subject_id', 'hadm_id'])
demo = demo.rename(columns={"icd_code": "proc_icd_code", "icd_version": "proc_icd_version", "seq_num": "proc_seq_num"})

# Group into higher order categories
def group_and_prefix_diag(code,version):
    return str(version) + "_" + code[:3]

def group_and_prefix_proc(code,version):
    s =  "IP_" + str(version) + "_" + str(code)[:3]
    return ''.join(s.split())

demo['diag_icd_code'] = demo.apply(lambda x: group_and_prefix_diag(x.diag_icd_code, x.diag_icd_version), axis = 1)
demo['proc_icd_code'] = demo.apply(lambda x: group_and_prefix_proc(x.proc_icd_code, x.proc_icd_version), axis = 1)
demo

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,admittime,dischtime,deathtime,...,outtime,los,admission_age,death,diag_seq_num,diag_icd_code,diag_icd_version,proc_seq_num,proc_icd_code,proc_icd_version
0,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 09:13:20,0.146065,30,0,4,9_296,9,5,IP_9_960,9
1,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 09:13:20,0.146065,30,0,4,9_296,9,4,IP_9_790,9
2,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 09:13:20,0.146065,30,0,4,9_296,9,3,IP_9_790,9
3,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 09:13:20,0.146065,30,0,4,9_296,9,2,IP_9_781,9
4,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 09:13:20,0.146065,30,0,4,9_296,9,1,IP_9_781,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4782716,19999068,M,63,2161,2011 - 2013,NaN,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,...,2161-08-30 23:48:04,6.765324,63,0,10,9_292,9,2,IP_9_967,9
4782717,19999068,M,63,2161,2011 - 2013,NaN,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,...,2161-08-30 23:48:04,6.765324,63,0,6,9_530,9,1,IP_9_960,9
4782718,19999068,M,63,2161,2011 - 2013,NaN,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,...,2161-08-30 23:48:04,6.765324,63,0,6,9_530,9,2,IP_9_967,9
4782719,19999068,M,63,2161,2011 - 2013,NaN,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,...,2161-08-30 23:48:04,6.765324,63,0,4,9_920,9,1,IP_9_960,9


In [3]:
t = pd.merge(pd.merge(pat,adm,on='subject_id'),ie,on=['subject_id', 'hadm_id'])
t

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,admittime,dischtime,deathtime,...,ethnicity,edregtime,edouttime,hospital_expire_flag,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,WHITE,2165-04-24 03:00:00,2165-04-24 05:43:00,0,35053963,Trauma SICU (TSICU),Trauma SICU (TSICU),2165-04-24 05:43:00,2165-04-24 09:13:20,0.146065
1,10004235,M,47,2196,2014 - 2016,NaN,24181354,2196-02-24 14:38:00,2196-03-04 14:02:00,NaN,...,BLACK/AFRICAN AMERICAN,2196-02-24 12:15:00,2196-02-24 17:07:00,0,30276431,Coronary Care Unit (CCU),Medical Intensive Care Unit (MICU),2196-02-24 17:07:00,2196-02-29 15:58:02,4.952106
2,10004720,M,61,2183,2014 - 2016,NaN,22081550,2186-11-12 18:01:00,2186-11-17 18:30:00,2186-11-17 18:30:00,...,WHITE,2186-11-12 16:09:00,2186-11-12 19:55:00,1,30274385,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2186-11-12 19:55:00,2186-11-17 21:15:55,5.056192
3,10005348,M,76,2128,2011 - 2013,NaN,25239799,2130-10-26 17:03:00,2130-11-02 16:00:00,NaN,...,WHITE,NaN,NaN,0,31523640,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2130-10-27 12:06:00,2130-10-29 12:05:02,1.999329
4,10006580,F,63,2137,2014 - 2016,NaN,24159665,2137-08-10 11:00:00,2137-08-15 13:25:00,NaN,...,HISPANIC/LATINO,NaN,NaN,0,37510899,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2137-08-10 16:23:10,2137-08-11 13:56:56,0.898449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69483,19992202,F,80,2153,2014 - 2016,NaN,20329411,2153-02-26 00:48:00,2153-03-02 17:00:00,NaN,...,WHITE,2153-02-25 18:11:00,2153-02-26 01:58:00,0,35275339,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2153-02-26 20:47:12,2153-02-27 09:11:40,0.516991
69484,19995244,F,80,2111,2017 - 2019,NaN,27127412,2111-01-11 00:00:00,2111-01-14 19:16:00,NaN,...,WHITE,NaN,NaN,0,37061220,Neuro Intermediate,Neuro Intermediate,2111-01-11 23:11:50,2111-01-14 00:17:36,2.045671
69485,19997448,F,52,2121,2014 - 2016,NaN,23560173,2122-07-22 14:00:00,2122-07-30 16:30:00,NaN,...,WHITE,NaN,NaN,0,37631835,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2122-07-23 10:35:21,2122-07-27 02:53:13,3.679074
69486,19997448,F,52,2121,2014 - 2016,NaN,23560173,2122-07-22 14:00:00,2122-07-30 16:30:00,NaN,...,WHITE,NaN,NaN,0,37114909,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2122-07-27 02:53:13,2122-07-28 12:33:32,1.402998


In [10]:
# Group codes into sequence
group_diag = demo.groupby(['subject_id','hadm_id'])['diag_icd_code'].apply(list)
group_proc = demo.groupby(['subject_id','hadm_id'])['proc_icd_code'].apply(list)
group_age = demo.groupby(['subject_id','hadm_id'])['admission_age'].apply(lambda x: list(x)[0])
group_gender = demo.groupby(['subject_id','hadm_id'])['gender'].apply(lambda x: list(x)[0])
group_ethnicity = demo.groupby(['subject_id','hadm_id'])['ethnicity'].apply(lambda x: list(x)[0])
group_death = demo.groupby(['subject_id','hadm_id'])['death'].apply(lambda x: list(x)[0])
group_los = demo.groupby(['subject_id','hadm_id'])['los'].apply(lambda x: list(x)[0])

temp_final = pd.concat([group_age, group_gender,group_ethnicity, group_los, group_diag, group_proc, group_death], axis=1).reset_index()

In [30]:
# Again, group codes into sequence, separated by patient id
group_diag_0 = temp_final.groupby(['subject_id'])['diag_icd_code'].apply(list)
group_proc_0 = temp_final.groupby(['subject_id'])['proc_icd_code'].apply(list)
group_age_0 = temp_final.groupby(['subject_id'])['admission_age'].apply(lambda x: list(x))
group_gender_0 = temp_final.groupby(['subject_id'])['gender'].apply(lambda x: list(x)[0])
group_death_0 = temp_final.groupby(['subject_id'])['death'].apply(lambda x: list(x)[0])
group_ethnicity_0 = temp_final.groupby(['subject_id'])['ethnicity'].apply(lambda x: list(x)[0])
group_los_0 = temp_final.groupby(['subject_id'])['los'].apply(list)
group_hadm_id_0 = temp_final.groupby(['subject_id'])['hadm_id'].apply(list)

final = pd.concat([group_hadm_id_0, group_age_0, group_gender_0 ,group_ethnicity_0 , group_los_0, group_diag_0, group_proc_0, group_death_0], axis=1).reset_index()
final = final.assign(long_los = 1 if final['los'][0][-1] > 9 else 0 )

In [51]:
# Sequentializes codes into sentences
def _sequentialize_codes(diag, proc):

    lst_len = len(diag)
    seqs = []
        
    for i in range(lst_len):
        
        # Filter unique values
        u_diag = list(set(diag[i]))
        u_proc = list(set(proc[i]))
        concat = u_diag + u_proc
        inner_seq = "<SEP>".join(concat)
        seqs.append(inner_seq)
    seq = ";".join(seqs)
    return seq

# Convert gender str to int
def _categorize_gender(gender):    
    return 0 if gender == "M" else 1

# Assign long los. This column is true if the last visit is longer than 9 days
def _long_los(los):    
    return 1 if los[-1] > 9 else 0

# Assign age group. This column is true if the patient is older than 65 y.o
def _is_old(age):    
    return 1 if age[-1] > 64 else 0

final['sequential_code'] = final.apply(lambda x: _sequentialize_codes(x.diag_icd_code, x.proc_icd_code), axis = 1)
final['gender'] = final.apply(lambda x: _categorize_gender(x.gender), axis = 1)
final['long_los'] = final.apply(lambda x: _long_los(x.los), axis = 1)
final['is_old'] = final.apply(lambda x: _is_old(x.admission_age), axis = 1)

In [54]:
final[(final['is_old'] == 0) & (final['death'] == 1)]

,subject_id,hadm_id,admission_age,gender,ethnicity,los,diag_icd_code,proc_icd_code,death,long_los,sequential_code,is_old
19,10004720,[22081550],[64],1,WHITE,[5.05619212962963],"[[10_Z66, 10_Z66, 10_Z66, 10_Z66, 10_J96, 10_J...","[[IP_10_0W9, IP_10_02H, IP_10_5A1, IP_10_3E0, ...",1,0,10_J96<SEP>10_L40<SEP>10_J69<SEP>10_M72<SEP>10...,0
27,10006053,[22942076],[52],1,UNKNOWN,[1.7513657407407408],"[[9_571, 9_571, 9_571, 9_459, 9_459, 9_459, 9_...","[[IP_9_389, IP_9_549, IP_9_391, IP_9_389, IP_9...",1,0,9_571<SEP>9_459<SEP>9_286<SEP>9_572<SEP>9_456<...,0
124,10035168,[26166196],[58],1,WHITE,[8.80917824074074],"[[9_484, 9_484, 9_484, 9_484, 9_484, 9_484, 9_...","[[IP_9_861, IP_9_990, IP_9_990, IP_9_960, IP_9...",1,0,9_288<SEP>9_482<SEP>9_785<SEP>9_484<SEP>9_511<...,0
134,10037975,[27617929],[60],1,UNKNOWN,[4.878240740740741],"[[9_356, 9_356, 9_356, 9_427, 9_427, 9_427, 9_...","[[IP_9_966, IP_9_389, IP_9_967, IP_9_966, IP_9...",1,0,9_V49<SEP>9_496<SEP>9_482<SEP>9_305<SEP>9_785<...,0
135,10038081,[20755971],[63],1,UNKNOWN,[3.698518518518519],"[[9_518, 9_518, 9_518, 9_518, 9_518, 9_518, 9_...","[[IP_9_210, IP_9_389, IP_9_451, IP_9_212, IP_9...",1,0,9_V49<SEP>9_403<SEP>9_V72<SEP>9_V66<SEP>9_789<...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34972,19959862,[22780970],[51],1,OTHER,[0.3794097222222222],"[[9_571, 9_571, 9_571, 9_571, 9_571, 9_572, 9_...","[[IP_9_967, IP_9_960, IP_9_389, IP_9_996, IP_9...",1,0,9_571<SEP>9_427<SEP>9_518<SEP>9_570<SEP>9_572<...,0
34998,19968075,[28592225],[61],1,UNKNOWN,[1.3971875],"[[10_Z98, 10_Z98, 10_Z98, 10_Z98, 10_Z98, 10_Z...","[[IP_10_02P, IP_10_5A1, IP_10_5A2, IP_10_027, ...",1,0,10_J96<SEP>10_N17<SEP>10_I11<SEP>10_I50<SEP>10...,0
35006,19970470,[23848017],[56],1,UNKNOWN,[2.20300925925926],"[[9_995, 9_995, 9_995, 9_V66, 9_V66, 9_V66, 9_...","[[IP_9_967, IP_9_399, IP_9_389, IP_9_967, IP_9...",1,0,9_V49<SEP>9_785<SEP>9_V66<SEP>9_570<SEP>9_V58<...,0
35041,19983009,[25448442],[57],1,ASIAN,[11.454409722222222],"[[10_J96, 10_J96, 10_J96, 10_A04, 10_A04, 10_A...","[[IP_10_5A1, IP_10_0BH, IP_10_3E0, IP_10_5A1, ...",1,1,10_A04<SEP>10_J96<SEP>10_R65<SEP>10_R64<SEP>10...,0


In [55]:
# Assign treatment. Treatment is true if patient received 'IP_9_9671' procedure
def assign_treatment(proc):

    lst_len = len(proc)
    seqs = []
        
    for i in range(lst_len):
        if 'IP_9_967' in proc[i]:
            return 1
    return 0

final["treated"] = final.apply(lambda x: assign_treatment(x.proc_icd_code), axis = 1)

In [56]:
final.to_csv(processed_data_dir + "sequential_mimic.csv", index=False)

In [66]:
# Return max sequence length
def max_seq_length(seqs):
    
    # Flatten input list
    flatten = [item for sublist in seqs for item in sublist]
    
    # Split seq into token
    lens = []
    for s in flatten:
        
        # Strip white splace
        stripped_s = ''.join(s.split())
        
        # Split by ;
        stripped_s = stripped_s.split(";")
        
        for strip in stripped_s:
            # Split by <SEP>
            s = strip.split("<SEP>")
            lens.append(len(s))
            
    max_len = max(lens)
    return max_len

# Return max sequence length
def max_visit_length(seqs):
    
    # Flatten input list
    flatten = [item for sublist in seqs for item in sublist]
    
    # Split seq into token
    lens = []
    for s in flatten:
        
        # Strip white splace
        stripped_s = ''.join(s.split())
        
        # Split by ;
        stripped_s = stripped_s.split(";")
        
        for strip in stripped_s:
            
            lens.append(len(stripped_s))
    max_len = max(lens)
    return max_len

seqs = final.sequential_code.str[0:-1].str.split(',').tolist()

print("Max seq len:", max_seq_length(seqs))
print("Max visit len:", max_visit_length(seqs))
print("Treatment count:", final["treated"].value_counts())

Max seq len: 66
Max visit len: 20
Treatment count: 0    30666
1     4436
Name: treated, dtype: int64


In [39]:
def assign_old(age):
    return sum(age)/len(age) > 60

final["old_patient"] = final.apply(lambda x: assign_old(x.admission_age), axis=1)

In [43]:
old = final[final.old_patient == True]
young = final[final.old_patient == False]

print(young.treated.value_counts())
print(old.treated.value_counts())

0    10526
1     2170
Name: treated, dtype: int64
0    19292
1     3114
Name: treated, dtype: int64


In [8]:
# Get LOS, remove outliers
demo_filter = demo[demo['los'] < 51]
los = demo_filter["los"].unique()

# Seperate into short/stay (long: los > 10 days)
demo_filter = demo_filter.assign(long_los = demo['los'] > 10)

# Filter out duplicates procedures per visit
demo_proc = demo_filter[['hadm_id', 'proc_icd_code', 'long_los']].drop_duplicates()
demo_diag = demo_filter[['hadm_id', 'diag_icd_code', 'long_los']].drop_duplicates()

In [9]:
print("Short los top 10 proc: \n", demo_proc[demo_proc['long_los'] == False]['proc_icd_code'].value_counts()[:10])
print("Long los top 10 proc: \n", demo_proc[demo_proc['long_los'] == True]['proc_icd_code'].value_counts()[:10])
print("Top 10 proc: \n", demo_proc['proc_icd_code'].value_counts()[:10])

#print("Short los top 10 diag: \n", demo_diag[demo_diag['long_los'] == False]['diag_icd_code'].value_counts()[:10])
#print("Long los top 10 diag: \n", demo_diag[demo_diag['long_los'] == True]['diag_icd_code'].value_counts()[:10])
#print("Top 10 diag: \n", demo_diag['diag_icd_code'].value_counts()[:10])
print(d_icd_proc[d_icd_proc['icd_code'] == 9671])

Short los top 10 proc: 
 IP_9_389     9222
IP_9_967     6755
IP_10_5A1    5924
IP_9_396     3977
IP_10_02H    3917
IP_9_960     3846
IP_9_966     3207
IP_9_885     3026
IP_9_372     2844
IP_9_361     2630
Name: proc_icd_code, dtype: int64
Long los top 10 proc: 
 IP_9_967     2196
IP_9_389     1953
IP_9_966     1909
IP_10_5A1    1341
IP_10_3E0    1283
IP_9_332     1260
IP_9_960     1211
IP_10_02H    1144
IP_10_0BH     820
IP_9_311      662
Name: proc_icd_code, dtype: int64
Top 10 proc: 
 IP_9_389     11175
IP_9_967      8951
IP_10_5A1     7265
IP_9_966      5116
IP_10_02H     5061
IP_9_960      5057
IP_9_396      4255
IP_10_3E0     3900
IP_9_885      3273
IP_9_332      3213
Name: proc_icd_code, dtype: int64
Empty DataFrame
Columns: [icd_code, icd_version, long_title]
Index: []


In [8]:
import tensorflow as tf 

x = tf.constant([[1., 0., 1., 0.], [1., 0., 1., 0.]])
print(tf.reduce_mean(x))

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (<ipython-input-8-5ad54a426b18>, line 3)

In [11]:
import numpy as np
x = np.array([[1., 0., 1., 0.], [1., 0., 1., 0.]])
print(x.shape)
print(np.mean(x))

(2, 4)
0.5
